In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import matplotlib.pyplot as plt

import folium
import rasterio
import rasterio.mask


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from geopy.distance import geodesic
import folium
from sklearn.tree import DecisionTreeClassifier

In [3]:
incoming_weather_size_draught = pd.read_csv('Data/Input_ML/incoming_weather_size_draught.csv')
incoming_weather_size = pd.read_csv('Data/Input_ML/incoming_weather_size.csv')
incoming_weather = pd.read_csv('Data/Input_ML/incoming_weather.csv')

In [6]:
incoming_weather

,Unnamed: 0,area_ID,geometry,index_right,from,from_rounded,from_location,to_haven,trip_ID,vessel_mmsi,Direction,Wind_speed
0,0,1107,"POLYGON ((4.065027429623431 51.98849659368361,...",1,2022-05-31 22:49:32+00:00,2022-05-31 23:00:00+00:00,"[4.06852, 51.98272]",MISS,0,370604000,SE,Low
1,1,1584,"POLYGON ((4.141337398344955 51.96305993744309,...",8,2022-05-31 23:34:58+00:00,2022-06-01 00:00:00+00:00,"[4.14163, 51.96557]",3PET,4,538005865,SE,Low
2,2,2453,MULTIPOLYGON (((4.284689496769295 51.893109132...,5,2022-05-31 23:39:58+00:00,2022-06-01 00:00:00+00:00,"[4.28209, 51.89865]",BOTL,2,538005077,SE,Low
3,3,1186,"POLYGON ((4.084104921803812 51.97577826556335,...",14,2022-06-01 01:11:15+00:00,2022-06-01 01:00:00+00:00,"[4.08005, 51.97787]",PET8,7,538007563,SW,Low
4,4,544,POLYGON ((3.9823582968417788 51.96305993744309...,11466,2022-06-01 01:16:11+00:00,2022-06-01 01:00:00+00:00,"[3.98159, 51.96625]",ALEXIA,6529,245508000,SW,Low
...,...,...,...,...,...,...,...,...,...,...,...,...
11145,11145,1146,"POLYGON ((4.071386593683558 51.98213742962348,...",42213,2023-05-31 17:43:21+00:00,2023-05-31 18:00:00+00:00,"[4.07534, 51.9803]",7PET,24481,257031520,N,Extreme
11146,11146,1938,"POLYGON ((4.204929038946226 51.92490495308232,...",42219,2023-05-31 21:01:36+00:00,2023-05-31 21:00:00+00:00,"[4.19908, 51.92873]",CKBRUG,24485,255806060,NE,Extreme
11147,11147,2414,"POLYGON ((4.28123900766775 51.8994682968418, 4...",42224,2023-05-31 21:17:18+00:00,2023-05-31 21:00:00+00:00,"[4.27635, 51.90111]",3PET,24489,538002883,NE,Extreme
11148,11148,1701,MULTIPOLYGON (((4.1667740545854635 51.94398244...,42231,2023-05-31 22:38:46+00:00,2023-05-31 23:00:00+00:00,"[4.16275, 51.9473]",7PET,24494,218085000,NE,High


In [4]:
leaving_weather_size_draught = pd.read_csv('Data/Input_ML/leaving_weather_size_draught.csv')
leaving_weather_size = pd.read_csv('Data/Input_ML/leaving_weather_size.csv')
leaving_weather = pd.read_csv('Data/Input_ML/leaving_weather.csv')

In [7]:
leaving_top = leaving_weather_size_draught['from_haven'].value_counts()
incoming_top = incoming_weather_size_draught['to_haven'].value_counts()
print(leaving_top[0:2])
print(incoming_top[0:2])

from_haven
EURO      451
AMALIA    309
Name: count, dtype: int64
to_haven
EURO    480
7PET    377
Name: count, dtype: int64


In [8]:
top_leaving = 'EURO'
top_incoming = 'EURO'

harbor_leaving = leaving_weather_size_draught[leaving_weather_size_draught['from_haven']== top_leaving]
harbor_incoming = incoming_weather_size_draught[incoming_weather_size_draught['to_haven']== top_incoming]

In [11]:
columns_leaving = [
    'from_haven',
    'Direction',
    'Wind_speed',
    'Lenght'	,
    'Width'	,
    'Type'
]

columns_incoming = [
    'to_haven',
    'Direction',
    'Wind_speed',
    'Lenght'	,
    'Width'	,
    'Type'
]


In [12]:
X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(harbor_leaving[columns_leaving], harbor_leaving['area_ID'], test_size=0.3, random_state=42)


In [13]:
# Initialize and train the decision tree regression model for latitude
decision_tree_leaving = DecisionTreeClassifier(random_state=42)
decision_tree_leaving.fit(X_from_lat_train, y_from_lat_train)

ValueError: could not convert string to float: 'EURO'